In [1]:
from Bio import Entrez
from Bio import SeqIO

In [2]:
#setting your email
Entrez.email='rakeshgartaula983@gmail.com'

In [32]:
#loading single fasta file for the given id
handle=Entrez.efetch(db='nucleotide',id='186972394',rettype='fasta')
record=SeqIO.read(handle,'fasta')
outputname='D:/ADSS_matL.fasta'


#for multiple fasta files with multiple ids- we need to parse the fasta sequences with SeqIO
#record=SeqIO.parse(handle,'fasta')

In [34]:
SeqIO.write(record,outputname,'fasta')

1

In [35]:
# -*- coding: utf-8 -*-
import pandas as pd
import re
import json
import requests
from contextlib import closing
import sqlite3

# Imprting prefect
import prefect
from prefect import task, Flow, Parameter


from elasticsearch import helpers


def fetch_data_from_id(id, query):
    print('Id', id)
    response = requests.get(
        'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore&retmode=xml&tool=rex-ncbi&id={}'.format(id))
    if response.status_code == 400:
        raise RuntimeError("Invalid PMID/Search Query ({})".format(query))
    return get_data_from_xml(response.content)

#@task
def fetch_data_from_query(query):
    query_response = requests.get(
            'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nuccore&retmode=json&tool=rex-ncbi&term=%s&retmax=100' % str(query)).json()
    nuccore_list = query_response['esearchresult']['idlist']     
    
    ID_LIST=[]
    all_data = []
    json_dataset = []
    ids_json = {}
    for i in nuccore_list:
        output = fetch_data_from_id(i, query)
        #print("-------------------------------------------------------")
        #print("this is ", nuccore_list)
        print(output)
        
        json_data = {}
        json_data['id'] = i
        json_data['title'] = output[0]
        json_data['source'] = output[1] 
        json_data['organism'] = output[2]
        json_data['taxonomy'] = output[3]
        ids_json[query] = nuccore_list
        json_dataset.append(json_data)
        all_data.append([i, output[0], output[1], output[2] , output[3]])
        ID_LIST.append([i])
        
    print("this is the id list",ID_LIST)
    return all_data



#@task
def fetch_id_from_query(query):
    query_response = requests.get(
            'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nuccore&retmode=json&tool=rex-ncbi&term=%s&retmax=100' % str(query)).json()
    nuccore_list = query_response['esearchresult']['idlist']     
    
    ID_LIST=[]
    for i in nuccore_list:
        output = fetch_data_from_id(i, query)
        #print("-------------------------------------------------------")
        #print("this is ", nuccore_list)
        print(output)
        
        json_data = {}
        json_data['id'] = i
        ID_LIST.append([i])
    print("--------------------------------")
    print("this is the id list",ID_LIST)
    return ID_LIST



#fetching only the id's from the nuccore db -- checpoint1

def fetch_data_from_id(id, query):
    print('Id', id)
    response = requests.get(
        'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore&retmode=xml&tool=rex-ncbi&id={}'.format(id))
    if response.status_code == 400:
        raise RuntimeError("Invalid PMID/Search Query ({})".format(query))
    return get_data_from_xml(response.content)




import xmltodict
def get_data_from_xml(response):
    print(response)
    # url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore&retmode=xml&tool=rex-ncbi&id=2230715355"
    # response = requests.get(url)
    data = xmltodict.parse(response)
    title = data['GBSet']['GBSeq']['GBSeq_definition']
    source = data['GBSet']['GBSeq']['GBSeq_source']
    organism = data['GBSet']['GBSeq']['GBSeq_organism']
    taxonomy = data['GBSet']['GBSeq']['GBSeq_taxonomy']

    res = [title, source,  organism, taxonomy]

    return res
    
    

#@task
def store_ncbi_data(parsed):
    create_script = 'CREATE TABLE IF NOT EXISTS nuccore_table (id INT, title TEXT, source TEXT, organism TEXT, taxonomy TEXT)'
    insert_cmd = "INSERT INTO nuccore_table VALUES (?, ?, ?, ?, ?)"
    select_cmd = "SELECT * FROM nuccore_table"


    with closing(sqlite3.connect("prefect_test.db")) as conn:
        with closing(conn.cursor()) as cursor:
            cursor.executescript(create_script)
            cursor.executemany(insert_cmd, parsed)
            a = cursor.execute(select_cmd)
            conn.commit()

            
            #Delete the duplicate data (deduplicate)
            cursor.execute('''DELETE FROM nuccore_table WHERE EXISTS 
            ( SELECT 1 FROM nuccore_table t2 WHERE nuccore_table.id = t2.id AND nuccore_table.rowid > t2.rowid)''')
            conn.commit()


            # Display data inserted -- test done for only 4 ids 
            print("Data Inserted in the table: ")
            data=cursor.execute('''SELECT * FROM nuccore_table''')
            for row in data:
                print(row)


#query = Parameter('NCBI', default='NCBI')
parsed = fetch_data_from_query('NCBI')
parsed_id = fetch_id_from_query('NCBI')
store_ncbi_data(parsed)
print("This is the final list of Id's",parsed_id)

   

Id 2240894608
b'<?xml version="1.0" encoding="UTF-8"  ?>\n<!DOCTYPE GBSet PUBLIC "-//NCBI//NCBI GBSeq/EN" "https://www.ncbi.nlm.nih.gov/dtd/NCBI_GBSeq.dtd">\n<GBSet>\n  <GBSeq>\n\n    <GBSeq_locus>JAILZC010000000</GBSeq_locus>\n    <GBSeq_length>322</GBSeq_length>\n    <GBSeq_strandedness>double</GBSeq_strandedness>\n    <GBSeq_moltype>DNA</GBSeq_moltype>\n    <GBSeq_topology>linear</GBSeq_topology>\n    <GBSeq_division>ENV</GBSeq_division>\n    <GBSeq_update-date>19-MAY-2022</GBSeq_update-date>\n    <GBSeq_create-date>19-MAY-2022</GBSeq_create-date>\n    <GBSeq_definition>MAG: Candidatus Bipolaricaulota bacterium isolate HOT.Cluster.56, whole genome shotgun sequencing project</GBSeq_definition>\n    <GBSeq_primary-accession>JAILZC000000000</GBSeq_primary-accession>\n    <GBSeq_accession-version>JAILZC000000000.1</GBSeq_accession-version>\n    <GBSeq_other-seqids>\n      <GBSeqid>gb|JAILZC000000000.1|JAILZC010000000</GBSeqid>\n      <GBSeqid>gi|2240894608</GBSeqid>\n    </GBSeq_other-s

b'<?xml version="1.0" encoding="UTF-8"  ?>\n<!DOCTYPE GBSet PUBLIC "-//NCBI//NCBI GBSeq/EN" "https://www.ncbi.nlm.nih.gov/dtd/NCBI_GBSeq.dtd">\n<GBSet>\n  <GBSeq>\n\n    <GBSeq_locus>JAILZD010000000</GBSeq_locus>\n    <GBSeq_length>679</GBSeq_length>\n    <GBSeq_strandedness>double</GBSeq_strandedness>\n    <GBSeq_moltype>DNA</GBSeq_moltype>\n    <GBSeq_topology>linear</GBSeq_topology>\n    <GBSeq_division>ENV</GBSeq_division>\n    <GBSeq_update-date>19-MAY-2022</GBSeq_update-date>\n    <GBSeq_create-date>19-MAY-2022</GBSeq_create-date>\n    <GBSeq_definition>MAG: Dehalococcoidia bacterium isolate HOT.Cluster.58, whole genome shotgun sequencing project</GBSeq_definition>\n    <GBSeq_primary-accession>JAILZD000000000</GBSeq_primary-accession>\n    <GBSeq_accession-version>JAILZD000000000.1</GBSeq_accession-version>\n    <GBSeq_other-seqids>\n      <GBSeqid>gb|JAILZD000000000.1|JAILZD010000000</GBSeqid>\n      <GBSeqid>gi|2240894606</GBSeqid>\n    </GBSeq_other-seqids>\n    <GBSeq_projec

b'<?xml version="1.0" encoding="UTF-8"  ?>\n<!DOCTYPE GBSet PUBLIC "-//NCBI//NCBI GBSeq/EN" "https://www.ncbi.nlm.nih.gov/dtd/NCBI_GBSeq.dtd">\n<GBSet>\n  <GBSeq>\n\n    <GBSeq_locus>JAILZB010000000</GBSeq_locus>\n    <GBSeq_length>461</GBSeq_length>\n    <GBSeq_strandedness>double</GBSeq_strandedness>\n    <GBSeq_moltype>DNA</GBSeq_moltype>\n    <GBSeq_topology>linear</GBSeq_topology>\n    <GBSeq_division>ENV</GBSeq_division>\n    <GBSeq_update-date>19-MAY-2022</GBSeq_update-date>\n    <GBSeq_create-date>19-MAY-2022</GBSeq_create-date>\n    <GBSeq_definition>MAG: Chloroflexi bacterium isolate HOT.Cluster.3, whole genome shotgun sequencing project</GBSeq_definition>\n    <GBSeq_primary-accession>JAILZB000000000</GBSeq_primary-accession>\n    <GBSeq_accession-version>JAILZB000000000.1</GBSeq_accession-version>\n    <GBSeq_other-seqids>\n      <GBSeqid>gb|JAILZB000000000.1|JAILZB010000000</GBSeqid>\n      <GBSeqid>gi|2240894605</GBSeqid>\n    </GBSeq_other-seqids>\n    <GBSeq_project>PRJ

b'<?xml version="1.0" encoding="UTF-8"  ?>\n<!DOCTYPE GBSet PUBLIC "-//NCBI//NCBI GBSeq/EN" "https://www.ncbi.nlm.nih.gov/dtd/NCBI_GBSeq.dtd">\n<GBSet>\n  <GBSeq>\n\n    <GBSeq_locus>JAILZG010000000</GBSeq_locus>\n    <GBSeq_length>172</GBSeq_length>\n    <GBSeq_strandedness>double</GBSeq_strandedness>\n    <GBSeq_moltype>DNA</GBSeq_moltype>\n    <GBSeq_topology>linear</GBSeq_topology>\n    <GBSeq_division>ENV</GBSeq_division>\n    <GBSeq_update-date>19-MAY-2022</GBSeq_update-date>\n    <GBSeq_create-date>19-MAY-2022</GBSeq_create-date>\n    <GBSeq_definition>MAG: Firmicutes bacterium isolate HOT.Cluster.78, whole genome shotgun sequencing project</GBSeq_definition>\n    <GBSeq_primary-accession>JAILZG000000000</GBSeq_primary-accession>\n    <GBSeq_accession-version>JAILZG000000000.1</GBSeq_accession-version>\n    <GBSeq_other-seqids>\n      <GBSeqid>gb|JAILZG000000000.1|JAILZG010000000</GBSeqid>\n      <GBSeqid>gi|2240894604</GBSeqid>\n    </GBSeq_other-seqids>\n    <GBSeq_project>PRJ

b'<?xml version="1.0" encoding="UTF-8"  ?>\n<!DOCTYPE GBSet PUBLIC "-//NCBI//NCBI GBSeq/EN" "https://www.ncbi.nlm.nih.gov/dtd/NCBI_GBSeq.dtd">\n<GBSet>\n  <GBSeq>\n\n    <GBSeq_locus>JAILZE010000000</GBSeq_locus>\n    <GBSeq_length>555</GBSeq_length>\n    <GBSeq_strandedness>double</GBSeq_strandedness>\n    <GBSeq_moltype>DNA</GBSeq_moltype>\n    <GBSeq_topology>linear</GBSeq_topology>\n    <GBSeq_division>ENV</GBSeq_division>\n    <GBSeq_update-date>19-MAY-2022</GBSeq_update-date>\n    <GBSeq_create-date>19-MAY-2022</GBSeq_create-date>\n    <GBSeq_definition>MAG: Candidatus Rokubacteria bacterium isolate HOT.Cluster.5, whole genome shotgun sequencing project</GBSeq_definition>\n    <GBSeq_primary-accession>JAILZE000000000</GBSeq_primary-accession>\n    <GBSeq_accession-version>JAILZE000000000.1</GBSeq_accession-version>\n    <GBSeq_other-seqids>\n      <GBSeqid>gb|JAILZE000000000.1|JAILZE010000000</GBSeqid>\n      <GBSeqid>gi|2240894603</GBSeqid>\n    </GBSeq_other-seqids>\n    <GBSeq

...

b'<?xml version="1.0" encoding="UTF-8"  ?>\n<!DOCTYPE GBSet PUBLIC "-//NCBI//NCBI GBSeq/EN" "https://www.ncbi.nlm.nih.gov/dtd/NCBI_GBSeq.dtd">\n<GBSet>\n  <GBSeq>\n\n    <GBSeq_locus>NZ_AYHB01000258</GBSeq_locus>\n    <GBSeq_length>3806</GBSeq_length>\n    <GBSeq_strandedness>double</GBSeq_strandedness>\n    <GBSeq_moltype>DNA</GBSeq_moltype>\n    <GBSeq_topology>linear</GBSeq_topology>\n    <GBSeq_division>CON</GBSeq_division>\n    <GBSeq_update-date>19-MAY-2022</GBSeq_update-date>\n    <GBSeq_create-date>19-MAY-2022</GBSeq_create-date>\n    <GBSeq_definition>Fusobacterium nucleatum YWH7199 contig0263, whole genome shotgun sequence</GBSeq_definition>\n    <GBSeq_primary-accession>NZ_AYHB01000258</GBSeq_primary-accession>\n    <GBSeq_accession-version>NZ_AYHB01000258.1</GBSeq_accession-version>\n    <GBSeq_other-seqids>\n      <GBSeqid>ref|NZ_AYHB01000258.1|</GBSeqid>\n      <GBSeqid>gnl|WGS:NZ_AYHB01|contig0263</GBSeqid>\n      <GBSeqid>gi|2240893198</GBSeqid>\n    </GBSeq_other-seqid

b'<?xml version="1.0" encoding="UTF-8"  ?>\n<!DOCTYPE GBSet PUBLIC "-//NCBI//NCBI GBSeq/EN" "https://www.ncbi.nlm.nih.gov/dtd/NCBI_GBSeq.dtd">\n<GBSet>\n  <GBSeq>\n\n    <GBSeq_locus>NZ_AYHB01000257</GBSeq_locus>\n    <GBSeq_length>609</GBSeq_length>\n    <GBSeq_strandedness>double</GBSeq_strandedness>\n    <GBSeq_moltype>DNA</GBSeq_moltype>\n    <GBSeq_topology>linear</GBSeq_topology>\n    <GBSeq_division>CON</GBSeq_division>\n    <GBSeq_update-date>19-MAY-2022</GBSeq_update-date>\n    <GBSeq_create-date>19-MAY-2022</GBSeq_create-date>\n    <GBSeq_definition>Fusobacterium nucleatum YWH7199 contig0262, whole genome shotgun sequence</GBSeq_definition>\n    <GBSeq_primary-accession>NZ_AYHB01000257</GBSeq_primary-accession>\n    <GBSeq_accession-version>NZ_AYHB01000257.1</GBSeq_accession-version>\n    <GBSeq_other-seqids>\n      <GBSeqid>ref|NZ_AYHB01000257.1|</GBSeqid>\n      <GBSeqid>gnl|WGS:NZ_AYHB01|contig0262</GBSeqid>\n      <GBSeqid>gi|2240893197</GBSeqid>\n    </GBSeq_other-seqids

b'<?xml version="1.0" encoding="UTF-8"  ?>\n<!DOCTYPE GBSet PUBLIC "-//NCBI//NCBI GBSeq/EN" "https://www.ncbi.nlm.nih.gov/dtd/NCBI_GBSeq.dtd">\n<GBSet>\n  <GBSeq>\n\n    <GBSeq_locus>NZ_AYHB01000256</GBSeq_locus>\n    <GBSeq_length>1035</GBSeq_length>\n    <GBSeq_strandedness>double</GBSeq_strandedness>\n    <GBSeq_moltype>DNA</GBSeq_moltype>\n    <GBSeq_topology>linear</GBSeq_topology>\n    <GBSeq_division>CON</GBSeq_division>\n    <GBSeq_update-date>19-MAY-2022</GBSeq_update-date>\n    <GBSeq_create-date>19-MAY-2022</GBSeq_create-date>\n    <GBSeq_definition>Fusobacterium nucleatum YWH7199 contig0261, whole genome shotgun sequence</GBSeq_definition>\n    <GBSeq_primary-accession>NZ_AYHB01000256</GBSeq_primary-accession>\n    <GBSeq_accession-version>NZ_AYHB01000256.1</GBSeq_accession-version>\n    <GBSeq_other-seqids>\n      <GBSeqid>ref|NZ_AYHB01000256.1|</GBSeqid>\n      <GBSeqid>gnl|WGS:NZ_AYHB01|contig0261</GBSeqid>\n      <GBSeqid>gi|2240893196</GBSeqid>\n    </GBSeq_other-seqid

b'<?xml version="1.0" encoding="UTF-8"  ?>\n<!DOCTYPE GBSet PUBLIC "-//NCBI//NCBI GBSeq/EN" "https://www.ncbi.nlm.nih.gov/dtd/NCBI_GBSeq.dtd">\n<GBSet>\n  <GBSeq>\n\n    <GBSeq_locus>NZ_AYHB01000255</GBSeq_locus>\n    <GBSeq_length>739</GBSeq_length>\n    <GBSeq_strandedness>double</GBSeq_strandedness>\n    <GBSeq_moltype>DNA</GBSeq_moltype>\n    <GBSeq_topology>linear</GBSeq_topology>\n    <GBSeq_division>CON</GBSeq_division>\n    <GBSeq_update-date>19-MAY-2022</GBSeq_update-date>\n    <GBSeq_create-date>19-MAY-2022</GBSeq_create-date>\n    <GBSeq_definition>Fusobacterium nucleatum YWH7199 contig0260, whole genome shotgun sequence</GBSeq_definition>\n    <GBSeq_primary-accession>NZ_AYHB01000255</GBSeq_primary-accession>\n    <GBSeq_accession-version>NZ_AYHB01000255.1</GBSeq_accession-version>\n    <GBSeq_other-seqids>\n      <GBSeqid>ref|NZ_AYHB01000255.1|</GBSeqid>\n      <GBSeqid>gnl|WGS:NZ_AYHB01|contig0260</GBSeqid>\n      <GBSeqid>gi|2240893195</GBSeqid>\n    </GBSeq_other-seqids

b'<?xml version="1.0" encoding="UTF-8"  ?>\n<!DOCTYPE GBSet PUBLIC "-//NCBI//NCBI GBSeq/EN" "https://www.ncbi.nlm.nih.gov/dtd/NCBI_GBSeq.dtd">\n<GBSet>\n  <GBSeq>\n\n    <GBSeq_locus>NZ_AYHB01000254</GBSeq_locus>\n    <GBSeq_length>531</GBSeq_length>\n    <GBSeq_strandedness>double</GBSeq_strandedness>\n    <GBSeq_moltype>DNA</GBSeq_moltype>\n    <GBSeq_topology>linear</GBSeq_topology>\n    <GBSeq_division>CON</GBSeq_division>\n    <GBSeq_update-date>19-MAY-2022</GBSeq_update-date>\n    <GBSeq_create-date>19-MAY-2022</GBSeq_create-date>\n    <GBSeq_definition>Fusobacterium nucleatum YWH7199 contig0259, whole genome shotgun sequence</GBSeq_definition>\n    <GBSeq_primary-accession>NZ_AYHB01000254</GBSeq_primary-accession>\n    <GBSeq_accession-version>NZ_AYHB01000254.1</GBSeq_accession-version>\n    <GBSeq_other-seqids>\n      <GBSeqid>ref|NZ_AYHB01000254.1|</GBSeqid>\n      <GBSeqid>gnl|WGS:NZ_AYHB01|contig0259</GBSeqid>\n      <GBSeqid>gi|2240893194</GBSeqid>\n    </GBSeq_other-seqids

In [36]:
new_id = [ y for x in list(parsed_id) for y in x]
new_id_pr=','.join(map(str,new_id))
print(new_id_pr)

2240894608,2240894606,2240894605,2240894604,2240894603,2240894602,2240894601,2240894593,2240894592,2240894591,2240894590,2240894589,2240894588,2240894583,2240894582,2240894581,2240894580,2240894574,2240894573,2240894572,2240894571,2240894570,2240894569,2240894568,2240894567,2240894566,2240894565,2240894564,2240894563,2240894562,2240894555,2240894553,2240894552,2240894551,2240894550,2240894549,2240894548,2240894547,2240894546,2240894545,2240894544,2240894509,2240894508,2240894507,2240894506,2240894505,2240894498,2240894485,2240894483,2240894482,2240894481,2240894480,2240894479,2240894478,2240894477,2240894475,2240894467,2240894466,2240894465,2240894464,2240894463,2240894462,2240894461,2240894460,2240894459,2240894458,2240894457,2240894456,2240894455,2240894454,2240894453,2240894452,2240894451,2240894450,2240894449,2240894448,2240893217,2240893216,2240893215,2240893214,2240893213,2240893212,2240893211,2240893210,2240893209,2240893208,2240893207,2240893206,2240893205,2240893204,2240893203

In [38]:
#multi fasta files --loading based on the list of ids
handle=Entrez.efetch(db='nucleotide',id=list(new_id_pr),rettype='fasta')
record=SeqIO.parse(handle,'fasta')
outputname='D:/NCBI_Nuccore/Fasta_Files_and_Pangolin/Nuccore_sequence.fasta'

In [39]:
print(new_id_pr)

2240894608,2240894606,2240894605,2240894604,2240894603,2240894602,2240894601,2240894593,2240894592,2240894591,2240894590,2240894589,2240894588,2240894583,2240894582,2240894581,2240894580,2240894574,2240894573,2240894572,2240894571,2240894570,2240894569,2240894568,2240894567,2240894566,2240894565,2240894564,2240894563,2240894562,2240894555,2240894553,2240894552,2240894551,2240894550,2240894549,2240894548,2240894547,2240894546,2240894545,2240894544,2240894509,2240894508,2240894507,2240894506,2240894505,2240894498,2240894485,2240894483,2240894482,2240894481,2240894480,2240894479,2240894478,2240894477,2240894475,2240894467,2240894466,2240894465,2240894464,2240894463,2240894462,2240894461,2240894460,2240894459,2240894458,2240894457,2240894456,2240894455,2240894454,2240894453,2240894452,2240894451,2240894450,2240894449,2240894448,2240893217,2240893216,2240893215,2240893214,2240893213,2240893212,2240893211,2240893210,2240893209,2240893208,2240893207,2240893206,2240893205,2240893204,2240893203

In [40]:
SeqIO.write(record,outputname,'fasta')

691